In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# CD to project code directory
import os
os.chdir('/content/drive/MyDrive/Group_Project/Project_Code')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopy
from geopy import distance
import math
import plotly.express as px
import haversine as hs

pd.set_option('display.max_columns', 3000)
pd.set_option('display.max_rows', 3000)

In [ ]:
# import pittsburgh school data
# documentation: https://nces.ed.gov/ccd/districtsearch/district_detail.asp?ID2=4219170

school_info_short = pd.read_excel('/content/drive/MyDrive/Group_Project/School_Data/ncesdata_BB69E9D.xlsx', header = 1, skiprows = [0,1,2,3])

# import redfin data
school_detailed = pd.read_excel('/content/drive/MyDrive/Group_Project/School_Data/schools_detailed_cleaned.xlsx')

# Prepping School_info_short for join

In [ ]:
display(school_info_short.head())

,NCES School ID,State School ID,NCES District ID,State District ID,Low Grade*,High Grade*,School Name,District,County Name*,Street Address,City,State,ZIP,ZIP 4-digit,Phone,Locale Code*,Locale*,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*,Directly Certified*
0,4.219170e+11,PA-102027451-8203,4219170.0,PA-102027451,6.0,12.0,Academy at Westinghouse,Pittsburgh SD,Allegheny County,1101 N. Murtland Avenue,Pittsburgh,PA,15208.0,NaN,(412)529-3940,11.0,City: Large,No,No,Yes,Yes,704.0,65.70,10.7,701.0,0.0,–
1,4.219170e+11,PA-102027451-0409,4219170.0,PA-102027451,9.0,12.0,Pittsburgh Allderdice HS,Pittsburgh SD,Allegheny County,2409 Shady Ave,Pittsburgh,PA,15217.0,2409.0,(412)529-4800,11.0,City: Large,No,Yes,Yes,No,1430.0,106.54,13.4,1427.0,0.0,–
2,4.219170e+11,PA-102027451-7616,4219170.0,PA-102027451,6.0,8.0,Pittsburgh Allegheny 6-8,Pittsburgh SD,Allegheny County,810 Arch Street,Pittsburgh,PA,15212.0,5202.0,(412)529-4115,11.0,City: Large,No,Yes,Yes,Yes,171.0,17.02,10.0,171.0,0.0,–
3,4.219170e+11,PA-102027451-7615,4219170.0,PA-102027451,PK,5.0,Pittsburgh Allegheny K-5,Pittsburgh SD,Allegheny County,810 Arch Street,Pittsburgh,PA,15212.0,5202.0,(412)529-4100,11.0,City: Large,No,Yes,Yes,Yes,641.0,42.52,15.1,617.0,0.0,–
4,4.219170e+11,PA-102027451-7878,4219170.0,PA-102027451,PK,8.0,Pittsburgh Arlington K-8,Pittsburgh SD,Allegheny County,800 RECTENWALD STREET,Pittsburgh,PA,15210.0,NaN,(412)529-4700,11.0,City: Large,No,No,Yes,Yes,382.0,37.17,10.3,379.0,0.0,–


In [ ]:
# clean state district ID to join on info from school_detailed later
school_info_short['State School ID'] = school_info_short['State School ID'].str.split("-").str[2]
school_info_short.head()

,NCES School ID,State School ID,NCES District ID,State District ID,Low Grade*,High Grade*,School Name,District,County Name*,Street Address,City,State,ZIP,ZIP 4-digit,Phone,Locale Code*,Locale*,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*,Directly Certified*
0,4.219170e+11,8203,4219170.0,PA-102027451,6.0,12.0,Academy at Westinghouse,Pittsburgh SD,Allegheny County,1101 N. Murtland Avenue,Pittsburgh,PA,15208.0,NaN,(412)529-3940,11.0,City: Large,No,No,Yes,Yes,704.0,65.70,10.7,701.0,0.0,–
1,4.219170e+11,0409,4219170.0,PA-102027451,9.0,12.0,Pittsburgh Allderdice HS,Pittsburgh SD,Allegheny County,2409 Shady Ave,Pittsburgh,PA,15217.0,2409.0,(412)529-4800,11.0,City: Large,No,Yes,Yes,No,1430.0,106.54,13.4,1427.0,0.0,–
2,4.219170e+11,7616,4219170.0,PA-102027451,6.0,8.0,Pittsburgh Allegheny 6-8,Pittsburgh SD,Allegheny County,810 Arch Street,Pittsburgh,PA,15212.0,5202.0,(412)529-4115,11.0,City: Large,No,Yes,Yes,Yes,171.0,17.02,10.0,171.0,0.0,–
3,4.219170e+11,7615,4219170.0,PA-102027451,PK,5.0,Pittsburgh Allegheny K-5,Pittsburgh SD,Allegheny County,810 Arch Street,Pittsburgh,PA,15212.0,5202.0,(412)529-4100,11.0,City: Large,No,Yes,Yes,Yes,641.0,42.52,15.1,617.0,0.0,–
4,4.219170e+11,7878,4219170.0,PA-102027451,PK,8.0,Pittsburgh Arlington K-8,Pittsburgh SD,Allegheny County,800 RECTENWALD STREET,Pittsburgh,PA,15210.0,NaN,(412)529-4700,11.0,City: Large,No,No,Yes,Yes,382.0,37.17,10.3,379.0,0.0,–


In [ ]:
# Code address into one column to feed into geopy
school_info_short["Address"] = school_info_short['Street Address'] + ", " + school_info_short['City'] + ", " + school_info_short['State'] + ", USA"
school_info_short["Address"].iloc[0] = "1101 N Murtland St, Pittsburgh, PA, USA"
school_info_short.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,NCES School ID,State School ID,NCES District ID,State District ID,Low Grade*,High Grade*,School Name,District,County Name*,Street Address,City,State,ZIP,ZIP 4-digit,Phone,Locale Code*,Locale*,Charter,Magnet*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Free Lunch*,Reduced Lunch*,Directly Certified*,Address
0,4.219170e+11,8203,4219170.0,PA-102027451,6.0,12.0,Academy at Westinghouse,Pittsburgh SD,Allegheny County,1101 N. Murtland Avenue,Pittsburgh,PA,15208.0,NaN,(412)529-3940,11.0,City: Large,No,No,Yes,Yes,704.0,65.70,10.7,701.0,0.0,–,"1101 N Murtland St, Pittsburgh, PA, USA"
1,4.219170e+11,0409,4219170.0,PA-102027451,9.0,12.0,Pittsburgh Allderdice HS,Pittsburgh SD,Allegheny County,2409 Shady Ave,Pittsburgh,PA,15217.0,2409.0,(412)529-4800,11.0,City: Large,No,Yes,Yes,No,1430.0,106.54,13.4,1427.0,0.0,–,"2409 Shady Ave, Pittsburgh, PA, USA"
2,4.219170e+11,7616,4219170.0,PA-102027451,6.0,8.0,Pittsburgh Allegheny 6-8,Pittsburgh SD,Allegheny County,810 Arch Street,Pittsburgh,PA,15212.0,5202.0,(412)529-4115,11.0,City: Large,No,Yes,Yes,Yes,171.0,17.02,10.0,171.0,0.0,–,"810 Arch Street, Pittsburgh, PA, USA"
3,4.219170e+11,7615,4219170.0,PA-102027451,PK,5.0,Pittsburgh Allegheny K-5,Pittsburgh SD,Allegheny County,810 Arch Street,Pittsburgh,PA,15212.0,5202.0,(412)529-4100,11.0,City: Large,No,Yes,Yes,Yes,641.0,42.52,15.1,617.0,0.0,–,"810 Arch Street, Pittsburgh, PA, USA"
4,4.219170e+11,7878,4219170.0,PA-102027451,PK,8.0,Pittsburgh Arlington K-8,Pittsburgh SD,Allegheny County,800 RECTENWALD STREET,Pittsburgh,PA,15210.0,NaN,(412)529-4700,11.0,City: Large,No,No,Yes,Yes,382.0,37.17,10.3,379.0,0.0,–,"800 RECTENWALD STREET, Pittsburgh, PA, USA"


In [ ]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
# Used code snippets from:
# https://towardsdatascience.com/geocode-with-python-161ec1e62b89

locator = Nominatim(user_agent= "school_geocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
school_info_short['location'] = school_info_short['Address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
school_info_short['point'] = school_info_short['location'].apply(lambda loc: tuple(loc.point[:2]) if loc else None)

In [ ]:
# Create sub data frame with just id, student teacher ratio, and point
school_locs = school_info_short[['State School ID', 'Student Teacher Ratio*', 'High Grade*', 'point']].copy()
display(school_locs.head())

,State School ID,Student Teacher Ratio*,High Grade*,point
0,8203,10.7,12.0,"(40.460495, -79.900447)"
1,0409,13.4,12.0,"(40.429384, -79.9200788832652)"
2,7616,10.0,8.0,"(40.4531525, -80.00880656929186)"
3,7615,15.1,5.0,"(40.4531525, -80.00880656929186)"
4,7878,10.3,8.0,"(40.40879926056033, -79.97985374031964)"


# Prepping School_detailed for join

In [ ]:
school_locs['State School ID'] = school_locs['State School ID'].astype(str)
school_detailed['Schl'] = school_detailed['Schl'].astype(str).str.zfill(4)
print(type(school_locs['State School ID'].iloc[0]))
print(school_locs.dtypes)

<class 'str'>
State School ID           object
Student Teacher Ratio*    object
High Grade*               object
point                     object
dtype: object


In [ ]:
display(school_detailed.head(5))

,DistrictName,Name,AUN,Schl,DataElement,DisplayValue
0,Mariana Bracetti Academy CS,Mariana Bracetti Academy CS,126513480,7672,Meeting Annual Academic Growth Expectations EL...,77
1,Mariana Bracetti Academy CS,Mariana Bracetti Academy CS,126513480,7672,Meeting Annual Academic Growth Expectations Ma...,60
2,Mariana Bracetti Academy CS,Mariana Bracetti Academy CS,126513480,7672,Meeting Annual Academic Growth Expectations Sc...,76
3,Mariana Bracetti Academy CS,Mariana Bracetti Academy CS,126513480,7672,Percent Graduates Post Secondary Education (Al...,0.492
4,Mariana Bracetti Academy CS,Mariana Bracetti Academy CS,126513480,7672,Percent Graduation 4-Year Cohort (All Student),0.918


In [ ]:
schools = list(school_locs['State School ID'])
print(schools)
small_detailed = school_detailed[school_detailed['Schl'].isin(schools)]
print(len(small_detailed))

['8203', '0409', '7616', '7615', '7878', '7886', '7889', '7501', '6019', '6915', '7868', '8106', '0326', '0412', '7913', '7692', '6022', '0330', '6811', '6024', '8204', '6026', '0400', '6028', '7880', '8255', '6042', '7870', '6044', '7871', '7881', '8108', '8110', '0363', '0351', '0365', '8105', '6047', '8300', '0416', '0395', '6664', '7742', '0379', '8107', '7693', '7888', '7916', '7362', '7039', '7884', '7876', '7694', '7877', '7885', '7502']
336


In [ ]:
small_detailed.head(10)

,DistrictName,Name,AUN,Schl,DataElement,DisplayValue
3252,Pittsburgh SD,Pittsburgh Allderdice HS,102027451,0409,Meeting Annual Academic Growth Expectations EL...,72
3253,Pittsburgh SD,Pittsburgh Allderdice HS,102027451,0409,Meeting Annual Academic Growth Expectations Ma...,Insufficient Sample
3254,Pittsburgh SD,Pittsburgh Allderdice HS,102027451,0409,Meeting Annual Academic Growth Expectations Sc...,50
3255,Pittsburgh SD,Pittsburgh Allderdice HS,102027451,0409,Percent Graduates Post Secondary Education (Al...,0.694
3256,Pittsburgh SD,Pittsburgh Allderdice HS,102027451,0409,Percent Graduation 4-Year Cohort (All Student),0.885
3257,Pittsburgh SD,Pittsburgh Allderdice HS,102027451,0409,Percent Graduation 5-Year Cohort (All Student),0.907
3258,Pittsburgh SD,Pittsburgh Allegheny 6-8,102027451,7616,Meeting Annual Academic Growth Expectations EL...,75
3259,Pittsburgh SD,Pittsburgh Allegheny 6-8,102027451,7616,Meeting Annual Academic Growth Expectations Ma...,50
3260,Pittsburgh SD,Pittsburgh Allegheny 6-8,102027451,7616,Meeting Annual Academic Growth Expectations Sc...,51
3261,Pittsburgh SD,Pittsburgh Allegheny 6-8,102027451,7616,Percent Graduates Post Secondary Education (Al...,Not Applicable


In [ ]:
school_locs.head()
school_locs["Score"] = 0.0
school_locs.head(6)

,State School ID,Student Teacher Ratio*,High Grade*,point,Score
0,8203,10.7,12.0,"(40.460495, -79.900447)",0.0
1,0409,13.4,12.0,"(40.429384, -79.9200788832652)",0.0
2,7616,10.0,8.0,"(40.4531525, -80.00880656929186)",0.0
3,7615,15.1,5.0,"(40.4531525, -80.00880656929186)",0.0
4,7878,10.3,8.0,"(40.40879926056033, -79.97985374031964)",0.0
5,7886,8.6,8.0,"(40.46851959183673, -79.96221122448979)",0.0


# Calculating School Score

In [ ]:
def calculateSchoolScore(school_locs, small_detailed):
  for x in range(len(school_locs)):
    id = school_locs['State School ID'].iloc[x]
    is_high = school_locs['High Grade*'].iloc[x] == 12.0
    info_df = small_detailed[small_detailed['Schl'] == id]
    if (is_high):
      total = 0.0
      count = 0
      for y in range(3,6):
        if (info_df['DisplayValue'].iloc[y] != "Insufficient Sample"):
          total += info_df['DisplayValue'].iloc[y]
          count += 1
      score = 0.0
      if (count != 0):
          score = total/count
      school_locs['Score'].iloc[x] = score*100
    else:
      total = 0.0
      count = 0
      for y in range(3):
        if (type(info_df['DisplayValue'].iloc[y]) == int):
          total += info_df['DisplayValue'].iloc[y]
          count += 1
      score = 0.0
      if (count != 0):
          score = total/count
      school_locs['Score'].iloc[x] = score

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
school_locs = school_locs[school_locs.Score > 10.0]
len(school_locs)
# must reverse lat/long
# school_locs['point'] = school_locs['point'].str[::-1]
display(school_locs.head())

,State School ID,Student Teacher Ratio*,High Grade*,point,Score
0,8203,10.7,12.0,"(40.460495, -79.900447)",60.033333
1,0409,13.4,12.0,"(40.429384, -79.9200788832652)",82.866667
2,7616,10.0,8.0,"(40.4531525, -80.00880656929186)",58.666667
3,7615,15.1,5.0,"(40.4531525, -80.00880656929186)",50.000000
4,7878,10.3,8.0,"(40.40879926056033, -79.97985374031964)",58.000000


# Loading House Pickle and Finding closest schools

In [ ]:
homes = pd.read_pickle('Redfin_pickle_new')

In [ ]:
homes['HS_Score'] = 0.0
homes['ES_MS_Score'] = 0.0
homes['Location'] = homes['Location'].str[::-1]
homes.head()

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),Neighborhood,Location,dt,Crime Score,HS_Score,ES_MS_Score
2,275000.0,4.0,4.0,1785.0,1306.0,1900.0,https://www.redfin.com/PA/Pittsburgh/45-Greele...,South Side,"(40.4229551, -79.9743145)",2021-10-01,35.380360,0.0,0.0
3,302500.0,5.0,2.0,3034.0,2178.0,1900.0,https://www.redfin.com/PA/Pittsburgh/259-45th-...,Lawrenceville,"(40.4711406, -79.9576202)",2021-06-01,25.490296,0.0,0.0
4,301000.0,3.0,1.5,1972.0,1306.0,1890.0,https://www.redfin.com/PA/Pittsburgh/431-Taylo...,Bloomfield,"(40.4605557, -79.9523315)",2021-10-01,26.678413,0.0,0.0
9,379900.0,2.0,1.5,1226.0,1742.0,1890.0,https://www.redfin.com/PA/Pittsburgh/159-1-2-3...,Lawrenceville,"(40.4679636, -79.9655419)",2021-09-01,17.730598,0.0,0.0
11,390000.0,3.0,3.0,1519.0,2178.0,1910.0,https://www.redfin.com/PA/Pittsburgh/4919-Hatf...,Lawrenceville,"(40.4780842, -79.9573142)",2020-12-01,15.909914,0.0,0.0


In [ ]:
def calc_school_scores(homes_df, schools_df):
  for x in range(len(homes_df)):
    home_loc = homes_df['Location'].iloc[x]
    dist_em = 1000
    dist_hs = 1000
    #print("Home ", x)
    for y in range(len(schools_df)):
      is_high = schools_df['High Grade*'].iloc[y] == 12.0
      school_loc = schools_df['point'].iloc[y]
      dist = distance.distance(home_loc, school_loc).miles
      #print("Dist 1: ", dist)
      if (is_high):
        #print("High School")
        if (dist < dist_hs):
          #print("Dist 2: ", dist)
          dist_hs = dist
          homes_df['HS_Score'].iloc[x] = schools_df['Score'].iloc[y]
      else:
        if (dist < dist_em):
          dist_em = dist
          homes_df['ES_MS_Score'].iloc[x] = schools_df['Score'].iloc[y]
    #print("Final EM dist:", dist_em)
    #print("Final HS dist:", dist_hs)
  return homes_df


In [ ]:
baby_homes = homes.head(20)
baby_schools = school_locs.head(50)
baby_homes_test = calc_school_scores(baby_homes, baby_schools)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
display(baby_homes_test.head())

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),Neighborhood,Location,dt,Crime Score,HS_Score,ES_MS_Score
2,275000.0,4.0,4.0,1785.0,1306.0,1900.0,https://www.redfin.com/PA/Pittsburgh/45-Greele...,South Side,"(40.4229551, -79.9743145)",2021-10-01,35.380360,95.233333,62.000000
3,302500.0,5.0,2.0,3034.0,2178.0,1900.0,https://www.redfin.com/PA/Pittsburgh/259-45th-...,Lawrenceville,"(40.4711406, -79.9576202)",2021-06-01,25.490296,58.833333,79.666667
4,301000.0,3.0,1.5,1972.0,1306.0,1890.0,https://www.redfin.com/PA/Pittsburgh/431-Taylo...,Bloomfield,"(40.4605557, -79.9523315)",2021-10-01,26.678413,58.833333,54.500000
9,379900.0,2.0,1.5,1226.0,1742.0,1890.0,https://www.redfin.com/PA/Pittsburgh/159-1-2-3...,Lawrenceville,"(40.4679636, -79.9655419)",2021-09-01,17.730598,58.833333,70.000000
11,390000.0,3.0,3.0,1519.0,2178.0,1910.0,https://www.redfin.com/PA/Pittsburgh/4919-Hatf...,Lawrenceville,"(40.4780842, -79.9573142)",2020-12-01,15.909914,90.800000,79.666667


In [ ]:
homes_school_scores = calc_school_scores(homes, school_locs)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
homes_school_scores.head()

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),Neighborhood,Location,dt,Crime Score,HS_Score,ES_MS_Score
2,275000.0,4.0,4.0,1785.0,1306.0,1900.0,https://www.redfin.com/PA/Pittsburgh/45-Greele...,South Side,"(40.4229551, -79.9743145)",2021-10-01,35.380360,95.233333,62.000000
3,302500.0,5.0,2.0,3034.0,2178.0,1900.0,https://www.redfin.com/PA/Pittsburgh/259-45th-...,Lawrenceville,"(40.4711406, -79.9576202)",2021-06-01,25.490296,58.833333,79.666667
4,301000.0,3.0,1.5,1972.0,1306.0,1890.0,https://www.redfin.com/PA/Pittsburgh/431-Taylo...,Bloomfield,"(40.4605557, -79.9523315)",2021-10-01,26.678413,58.833333,54.500000
9,379900.0,2.0,1.5,1226.0,1742.0,1890.0,https://www.redfin.com/PA/Pittsburgh/159-1-2-3...,Lawrenceville,"(40.4679636, -79.9655419)",2021-09-01,17.730598,58.833333,70.000000
11,390000.0,3.0,3.0,1519.0,2178.0,1910.0,https://www.redfin.com/PA/Pittsburgh/4919-Hatf...,Lawrenceville,"(40.4780842, -79.9573142)",2020-12-01,15.909914,90.800000,79.666667


In [ ]:
homes_school_scores.to_pickle('Redfin_with_schools_2')

# Visualizing School Quality

Homes vs HS Quality

In [ ]:
fig = px.scatter_geo(data_frame = homes_school_scores, lat = homes_school_scores.Location.str[0], lon = homes_school_scores.Location.str[1], color = "HS_Score",
                    scope = 'usa', fitbounds = "locations")
fig.show()

Homes vs Elementary and Middle School Quality

In [ ]:
fig = px.scatter_geo(data_frame = homes_school_scores, lat = homes_school_scores.Location.str[0], lon = homes_school_scores.Location.str[1], color = "ES_MS_Score",
                    scope = 'usa', fitbounds = "locations")
fig.show()

Schools vs HS Quality

In [ ]:
fig = px.scatter_geo(data_frame = school_locs, lat = school_locs.point.str[0], lon = school_locs.point.str[1], color = "Score",
                    scope = 'usa', fitbounds = "locations")
fig.show()